In [1]:
import wfdb as wb
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

In [2]:
%matplotlib qt

# Get list of records

In [3]:
files = list(set([i[:-4] for i in os.listdir('../data/raw') if i.find('hea')!=-1]))

# Upload a file

In [11]:
sig_length = 3600

labels_list = [
 'A', 
 'E', 
 'F', 
 'J', 
 'L', 
 'N', 
  'R', 
 'S',
  'V', 
 'a', 
 'e', 
 'f', 
 'j', 
 'x', 
 '|', 
              ]

classes = []

for file in tqdm(files):

    signal = wb.rdsamp('../data/raw/'+file)
    fs = signal[1]['fs']

    sig_name = np.array(signal[1]['sig_name'])
    
    try:
        signal = signal[0][:,np.where(sig_name == 'MLII')[0][0]]
    except:
        continue


    annotation = wb.rdann('../data/raw/'+file,'atr').__dict__
    annotation_samples = annotation['sample']
    annotation = annotation['symbol']
    annotation = np.array(annotation)
    
    

    
    for i in range(signal.shape[0]//sig_length):
        label_arr = 1
        temp_signal = signal[i*sig_length:(i+1)*sig_length]
        
        temp_annotation = annotation[np.where((annotation_samples > i*sig_length) & (annotation_samples < (i+1)*sig_length))]
        
        if temp_annotation[np.where(temp_annotation == 'N')].shape[0]>0:
            label_arr = 0
        
        
        label = {'label':[label_arr]}
        label['letters'] = temp_annotation.tolist()
        #save files
        os.makedirs('../data/formatted/'+file, exist_ok=True)
        np.save('../data/formatted/'+file+f'/{i}.npy',temp_signal)
        with open('../data/formatted/'+file+f'/{i}.json', 'w') as outfile:
            json.dump(label, outfile)
    
    
    

100%|██████████| 48/48 [00:05<00:00,  8.35it/s]
